# BT4103 Data Cleaning
Please read through and let me know if there are any issues with regard to the cleaning.

In [ ]:
import sys
!{sys.executable} -m pip install pandas scikit-learn

## Import packages and datasets
I have imported all the packages that I used up here for ease of reference. Please add your own filepath so that you can import the data correctly.

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re
import unicodedata

### Importing dataset
Add filepath below

In [ ]:
data = pd.read_excel("oots-cleaned-unlocked.xlsx")
data.head()

## Define Helper Functions
Here are the helper functions that I have created for easier readability in the actual code below.

In [ ]:
def OHE(df, col, drop_first=False): #One Hot Encode a column in a df
    dummies = pd.get_dummies(df[col], prefix=col, drop_first=drop_first)
    df = pd.concat([df.drop(columns=[col]), dummies], axis=1)
    legend = {new_col: category 
              for new_col, category in zip(dummies.columns, dummies.columns.str.replace(f"{col}_", "", regex=False))}
    return df, legend


def LabelEncode(df, col): #Label Encode a column in a df
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    legend = dict(zip(le.classes_, le.transform(le.classes_)))
    return df, legend

def inspect_column(df, col, top_n=20): #Get Unique Value information on column in df
    print(f"Column: {col}")
    print(f"Unique values: {df[col].nunique(dropna=False)}")
    print("\nTop value counts:")
    print(df[col].value_counts(dropna=False).head(top_n))

def apply_mapping(df, col, mapping, new_col_suffix="_clean"): #Apply new mapping to column in df
    new_col = col + new_col_suffix
    df[new_col] = df[col].replace(mapping)
    return df

def normalize_text(df, col, to_lower=True, replace_symbols=True, unknown_vals=None): #Normalise text of a column in a df
    s = df[col].astype(str).str.strip()
    if to_lower:
        s = s.str.lower()
    s = s.str.replace(r"\s+", " ", regex=True)
    if replace_symbols:
        s = s.str.replace(r"[-_/]", " ", regex=True)
    if unknown_vals:
        s = s.replace(unknown_vals, "0")
    df[col] = s
    return df

def _normalize_text_series(s: pd.Series) -> pd.Series: #Normalise text in series
    s = s.astype(str).map(lambda x: unicodedata.normalize("NFKC", x))
    s = s.str.strip()
    s = s.str.lower()
    s = s.str.replace(r"\s+", " ", regex=True)         
    s = s.str.replace(r"\s*/\s*", " / ", regex=True)   
    s = s.str.replace(r"\s*,\s*", ", ", regex=True)    
    s = s.str.strip(" ,")                              
    return s

def _remove_trailing_code_in_parens(name_s: pd.Series, code_s: pd.Series) -> pd.Series: #remove white spaces in ()
    code_up = code_s.astype(str).str.strip().str.upper()
    pattern = r"\(\s*{}\s*\)\s*$"
    out = name_s.copy()
    mask = code_up.notna() & code_up.ne("")
    out.loc[mask] = [
        re.sub(pattern.format(re.escape(c)), "", n, flags=re.IGNORECASE)
        for n, c in zip(out.loc[mask].tolist(), code_up.loc[mask].tolist())
    ]
    return out.str.strip(" ,")

def _choose_canonical_name(name_series: pd.Series) -> str: #Chooses best name
    s = name_series.dropna().astype(str)
    s = s[s.str.strip().ne("").values]
    s = s[s.str.strip().ne("unknown").values]
    if s.empty:
        return "0"
    vc = s.value_counts()
    top_freq = vc.iloc[0]
    candidates = vc[vc.eq(top_freq)].index.tolist()
    return max(candidates, key=len)

def build_operation_legend_and_drop_nature( #Nature cleaning
    df: pd.DataFrame,
    code_col: str = "OPERATION_CODE",
    nature_col: str = "NATURE",
    drop_nature: bool = True,
    keep_title_case_copy: bool = False,
    unknown_tokens = ("0", "na", "n/a", "-", "null", "nan")
):
    if code_col not in df.columns or nature_col not in df.columns:
        raise KeyError(f"Expected columns '{code_col}' and '{nature_col}' in df.")
    work = pd.DataFrame({
        "operation_code": df[code_col].astype(str).str.strip().str.upper(),
        "operation_name_raw": df[nature_col]
    })
    name_norm = _normalize_text_series(work["operation_name_raw"])
    name_norm = name_norm.replace(list(unknown_tokens), "unknown")
    name_clean = _remove_trailing_code_in_parens(name_norm, work["operation_code"])
    tmp = pd.DataFrame({"operation_code": work["operation_code"], "operation_name": name_clean})
    tmp = tmp[tmp["operation_code"].str.len() > 0]
    legend = (
        tmp.groupby("operation_code", as_index=False)["operation_name"]
           .apply(_choose_canonical_name)
           .rename(columns={"operation_name": "operation_name"})
    )
    if keep_title_case_copy:
        legend["operation_name_title"] = legend["operation_name"].str.title()
    df_out = df.copy()
    if drop_nature:
        df_out.drop(columns=[nature_col], inplace=True, errors="ignore")
    return df_out, legend

  
def clean_equipment( #EQUIPMENT cleaning
    df,
    col="EQUIPMENT",
    sep=";",
    tags_to_strip=(r"#nuh",),            
    unknown_vals=("0","na","n/a","-","null","nan",""),
    synonym_map=None         
):
    if synonym_map is None:
        synonym_map = {}
    pattern = r"|".join(fr"{re.escape(tag)}[_-]?" for tag in tags_to_strip)
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(pattern, "", regex=True, flags=re.IGNORECASE)
    )
    df = normalize_text(df, col, to_lower=True, replace_symbols=True, unknown_vals=unknown_vals)
    def _clean_token(tok: str) -> str:
        t = tok.strip()
        if not t: return ""
        t = re.sub(r"\s+", " ", t)
        t = synonym_map.get(t, t)
        if t in ("unknown",): return ""
        return t
    def _process_cell(cell: str) -> str:
        parts = re.split(rf"\s*{re.escape(sep)}\s*", cell) if cell else []
        cleaned = [_clean_token(p) for p in parts]
        cleaned = [c for c in cleaned if c]
        if not cleaned:
            return "unknown"
        cleaned = sorted(set(cleaned))
        return f"{sep} ".join(cleaned)
    df[col] = df[col].apply(_process_cell)
    return df

## General Cleaning
We are dropping index and case number as they are labels, and dropping patient name because it has been completely removed.

In [ ]:
data = data.drop(columns="INDEX")
data = data.drop(columns="PATIENT_NAME")
data = data.drop(columns="CASE_NUMBER")
data.head()

## Ensure correct data types
yet to do, still waiting on update from joey regarding the dates

In [ ]:
data.info()

## Handle Missing Data
Handling missing data is important to ensure that we do not run into any issues with the EDA, as well as our AI/ML implementations. It is a vital step in data cleaning to ensure that the dataset can be used efficiently and properly.

### Handling optional columns
These columns potentially will be blank as there is nothing to write for some operations, hence we will fill those with 0.

In [ ]:
data["Delay_Reason"].fillna(0, inplace=True)
data["Remarks"].fillna(0, inplace=True)
data["IMPLANT"].fillna(0, inplace=True)
data["EQUIPMENT"].fillna(0, inplace=True)
data["EMERGENCY_PRIORITY"].fillna(0, inplace=True)

### Handling missing date/time columns
These columns are missing certain timestamps. Hence, for those with missing timestamps, we will fill in "1900-01-01 00:00:00" so that we can maintain the date/time format while showing that it is obviously missing because it is in year 1900.

In [ ]:
time_cols = [c for c in data.columns if "TIME" in c.upper()]
extra_time_cols = [
    "PLANNED_ANAESTHESIA_INDUCTION",
    "ACTUAL_ANAESTHESIA_INDUCTION",
    "PLANNED_SKIN_CLOSURE",
    "ACTUAL_SKIN_CLOSURE",
    "BOOKING_DATE"
]

time_cols = list(set(time_cols + extra_time_cols))

placeholder = "1900-01-01 00:00:00"
data[time_cols] = data[time_cols].fillna(placeholder)

### Handle admission related columns
Some of these surgeries may be day surgeries of from the A&E, hence might not have admission data. Hence, we will replace blanks with "Not Admitted".

In [ ]:
admission_cols = ["ADMISSION_STATUS", "ADMISSION_CLASS_TYPE", 
                  "ADMISSION_TYPE", "ADMISSION_WARD", "ADMISSION_BED"]
data[admission_cols] = data[admission_cols].fillna("Not Admitted")

### Fill in missing staff data
Some surgeries are missing surgeon, anaesthetist, or diagnosis data, hence we will fill it with "Unknown" and "Not Recorded". This is because it is likely not possible for a surgery to proceed without them.

In [ ]:
clinician_cols = ["SURGEON", "ANAESTHETIST_TEAM", "ANAESTHETIST_MCR_NO"]
data[clinician_cols] = data[clinician_cols].fillna("Unknown")
data["DIAGNOSIS"] = data["DIAGNOSIS"].fillna("Not Recorded")

### Drop remaining missing rows
After filling in the missing values that we are able to fill, there are some columns that are still missing data. We will thus drop them as they make up a very small portion of our overall data.

In [ ]:
data.dropna(inplace=True)

### View current state of dataframe
Currently, the dataset no longer contains any missing data, and thus we are able to proceed with the next steps.

In [ ]:
data.info()

## Handle Duplicate Data
Important to remove to prevent bias in our AI/ML solution

### Check for duplicate rows
This is to see if our dataset contains any rows that are completely identical. This means that the same surgery has been accidentally logged twice. We want to avoid having this in our dataset as it would cause our analysis in the future to skew.

In [ ]:
data.duplicated().sum()

### Drop duplicate rows
We identified 3 duplicate rows, and hence we will want to drop them. 

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.info()

## Deep cleaning each column
Looking into each individual column to clean up most of the free text portions. Please add more cleaning as we go, as there is quite alot to sieve through and I dont think i caught it all.

In [ ]:
data.info()

### Inspect Location
No problems.

#### Unique values analysis

In [ ]:
inspect_column(data, "LOCATION", top_n=30)

### Inspect Room
No problems.

#### Unique values analysis

In [ ]:
inspect_column(data, "ROOM", top_n=30)

### Inspect case status
No problems.

#### Unique values analysis

In [ ]:
inspect_column(data, "CASE_STATUS", top_n=30)

### Inspect OPERATION_TYPE
No problems.

#### UVA

In [ ]:
inspect_column(data, "OPERATION_TYPE", top_n=30)

### Inspect Emergency Priority
No problems.

#### UVA

In [ ]:
inspect_column(data, "EMERGENCY_PRIORITY", top_n=30)

### Inspect Patient Code
No problems.

#### UVA

In [ ]:
inspect_column(data, "PATIENT_CODE", top_n=30)

### Inspect Nature
Removed this column entirely, and created a legend(can be found below) to map SURGICAL_CODE to NATURE, as they are the same thing.

#### UVA

In [ ]:
inspect_column(data, "NATURE", top_n=30)

In [ ]:
data, nature_legend = build_operation_legend_and_drop_nature(
    data,
    code_col="SURGICAL_CODE",
    nature_col="NATURE",
    drop_nature=True,           
    keep_title_case_copy=True    
)

In [ ]:
print(data.columns)              
nature_legend.drop(columns='operation_name_title', inplace=True)
nature_legend

In [ ]:
data.info()

### Inspect Surgical Code
Extension of NATURE.

#### UVA

In [ ]:
inspect_column(data, "SURGICAL_CODE", top_n=30)

### Inspect discipline
No problems.

#### UVA

In [ ]:
inspect_column(data, "DISCIPLINE", top_n=30)

### Inspect Surgeon
No problems.

#### UVA

In [ ]:
inspect_column(data, "SURGEON", top_n=30)

### Inspect ANAESTHETIST_TEAM
No problems.

#### UVA

In [ ]:
inspect_column(data, "ANAESTHETIST_TEAM", top_n=30)

### INSPECT ANAESTHETIST_MCR_NO
No problems.

#### UVA

In [ ]:
inspect_column(data, "ANAESTHETIST_MCR_NO", top_n=30)

### INSPECT ANESTHESIA
No problems.

#### UVA

In [ ]:
inspect_column(data, "ANESTHESIA", top_n=30)

### Inspect EQUIPMENT
Removed #NUH_ or #NUH from all entries, as well as alphabetically ordered the equipment such that even if they were in different orders, they would appear under the same unique value.

#### UVA

In [ ]:
inspect_column(data, "EQUIPMENT", top_n=30)

In [ ]:
synonyms = {}
data = clean_equipment(
    data,
    col="EQUIPMENT",
    tags_to_strip=(r"#nuh",),        
    unknown_vals=("0","na","n/a","-","null","nan",""),
    synonym_map=synonyms
)

# Inspect results
inspect_column(data, "EQUIPMENT", top_n=30)

### Inspect ADMISSION_STATUS
No problems.

#### UVA

In [ ]:
inspect_column(data, "ADMISSION_STATUS", top_n=30)

### Inspect ADMISSION_CLASS_TYPE
No problems.

#### UVA

In [ ]:
inspect_column(data, "ADMISSION_CLASS_TYPE", top_n=30)

### Inspect ADMISSION_TYPE
No problems.

#### UVA

In [ ]:
inspect_column(data, "ADMISSION_TYPE", top_n=30)

### Inspect ADMISSION_WARD
No problems.

#### UVA

In [ ]:
inspect_column(data, "ADMISSION_WARD", top_n=30)

### Inspect ADMISSION_BED
No problems.

#### UVA

In [ ]:
inspect_column(data, "ADMISSION_BED", top_n=30)

### Inspect AOH
Fixed True False.

#### UVA

In [ ]:
inspect_column(data, "AOH", top_n=30)

In [ ]:
data = normalize_text(data, "AOH", unknown_vals=["0", "na", "n/a", "-", "null", "nan"])
inspect_column(data, "AOH", top_n=30)

### Inspect BLOOD
No problems.

#### UVA

In [ ]:
inspect_column(data, "BLOOD", top_n=30)

### Inspect IMPLANT
remove 'yes' remove 'x1' remove multiple white spaces, leading and trailing whitespaces and symbols.

#### UVA

In [ ]:
inspect_column(data, "IMPLANT", top_n=30)

In [ ]:
data = normalize_text(data, "IMPLANT", unknown_vals=["0", "na", "n/a", "-", "null", "nan", "", "nil", "nil."])
data["IMPLANT"] = (
    data["IMPLANT"]
    .astype(str)
    .str.strip(" ;,.-")
    .str.replace(r"\bx\d+\b", "", regex=True)
    .str.replace(r"\byes\b", "", regex=True)
    .str.replace(r"\s+", " ", regex=True)    
    .str.strip()
    .str.replace("&", "and", regex=False)
)
inspect_column(data, "IMPLANT", top_n=30)

### Inspect DIAGNOSIS
just normal standardisation

#### UVA

In [ ]:
inspect_column(data, "DIAGNOSIS", top_n=30)

In [ ]:
data = normalize_text(data, "DIAGNOSIS", unknown_vals=["0", "na", "n/a", "-", "null", "nan", "", "nil"])
inspect_column(data, "DIAGNOSIS", top_n=100)

### Inspect CANCER_INDICATOR
just normal standardisation

#### UVA

In [ ]:
inspect_column(data, "CANCER_INDICATOR", top_n=30)

In [ ]:
data = normalize_text(data, "CANCER_INDICATOR", unknown_vals=["0", "na", "n/a", "-", "null", "nan"])
inspect_column(data, "CANCER_INDICATOR", top_n=30)

### Inspect TUMOR_INDICATOR
just normal standardisation

#### UVA

In [ ]:
inspect_column(data, "TRAUMA_INDICATOR", top_n=30)

In [ ]:
data = normalize_text(data, "TRAUMA_INDICATOR", unknown_vals=["0", "na", "n/a", "-", "null", "nan"])
inspect_column(data, "TRAUMA_INDICATOR", top_n=30)


### Inspect Delay_Reason
No problems.

#### UVA

In [ ]:
inspect_column(data, "Delay_Reason", top_n=30)

### Inspect Remarks
No problems.

#### UVA

In [ ]:
inspect_column(data, "Remarks", top_n=30)

Save file from cleaning steps above into a seperate file (change file name if required)

In [ ]:
output_path = "oots-data-cleaning-1.xlsx"
data.to_excel(output_path, index=False)

print(f"Saved to {output_path}")

In [ ]:
import re
import string
import pandas as pd
from collections import Counter

Find the highest frequency of words, bigrams, and trigrams to be used in taxonomy for categorisation

In [ ]:
INPUT_XLSX = "oots-data-cleaning-1.xlsx"

DELAY_COL = "Delay_Reason"

OUT_WORDS    = "oots-delay-wordcount-cleaned-1.csv"
OUT_BIGRAMS  = "oots-delay-bigram-count-cleaned-1.csv"
OUT_TRIGRAMS = "oots-delay-trigram-count-cleaned-1.csv"

In [ ]:
df = pd.read_excel(INPUT_XLSX)

if DELAY_COL not in df.columns:
    raise KeyError(f"'{DELAY_COL}' not found. Available columns: {list(df.columns)}")

df.head()

Normalize text in Delay_Reason (remove punctuation, standardise case, remove trialing spaces)

In [ ]:
_punct_tbl = str.maketrans("", "", string.punctuation)

def normalize_text(s: str) -> str:

    s = str(s).lower()
    s = s.translate(_punct_tbl)
    s = re.sub(r"\s+", " ", s).strip()

    s = re.sub(r"\bo\.t\b", "operating theater", s)
    s = re.sub(r"\bot\b", "operating theater", s)
    s = re.sub(r"\bo\.r\b", "operating room", s)
    s = re.sub(r"\banaesth\b", "anaesthesia", s)
    s = re.sub(r"\banesth\b", "anaesthesia", s)
    s = re.sub(r"\bpt\b", "patient", s)
    s = re.sub(r"\bprev\b", "previous", s)
    s = re.sub(r"\bdr\b", "doctor", s)
    s = re.sub(r"\bpre-med\b", "premedication", s)

    return s

# Apply normalization
df["_Delay_norm"] = df[DELAY_COL].astype(str).fillna("").map(normalize_text)
df[["_Delay_norm"]].head(10)

In [ ]:
STOPWORDS = {
    "the","a","an","and","or","to","of","for","by","with","from",
    "is","are","was","were","be","been","being","due","because",
    "this","that","it","as","into","per","via", "eg", "etc"
}

# Initialize containers
words, bigrams, trigrams = [], [], []

# Tokenize each delay reason
for text in df["_Delay_norm"]:
    tokens = [t for t in text.split() if t and t not in STOPWORDS]
    if not tokens:
        continue

    words.extend(tokens)
    if len(tokens) >= 2:
        bigrams.extend([" ".join(tokens[i:i+2]) for i in range(len(tokens)-1)])
    if len(tokens) >= 3:
        trigrams.extend([" ".join(tokens[i:i+3]) for i in range(len(tokens)-2)])

word_counts    = Counter(words)
bigram_counts  = Counter(bigrams)
trigram_counts = Counter(trigrams)


In [ ]:
pd.DataFrame(word_counts.most_common(),   columns=["Word",   "Count"]).to_csv(OUT_WORDS,   index=False, encoding="utf-8-sig")
pd.DataFrame(bigram_counts.most_common(), columns=["Bigram", "Count"]).to_csv(OUT_BIGRAMS, index=False, encoding="utf-8-sig")
pd.DataFrame(trigram_counts.most_common(),columns=["Trigram","Count"]).to_csv(OUT_TRIGRAMS,index=False, encoding="utf-8-sig")

print(f"Saved:\n - {OUT_WORDS}\n - {OUT_BIGRAMS}\n - {OUT_TRIGRAMS}")

Print top, words, bigrams, trigrams (change file according to updates)

In [ ]:
top_n = 30

print("\nTop Words:")
display(pd.DataFrame(word_counts.most_common(top_n), columns=["Word", "Count"]))

print("\nTop Bigrams:")
display(pd.DataFrame(bigram_counts.most_common(top_n), columns=["Bigram", "Count"]))

print("\nTop Trigrams:")
display(pd.DataFrame(trigram_counts.most_common(top_n), columns=["Trigram", "Count"]))


In [ ]:
import numpy as np

INPUT_FILE  = "oots-data-cleaning-1.xlsx"
OUTPUT_FILE = "oots-data-cleaning-3-flagged.xlsx"
OUTPUT_CSV  = "oots-data-cleaning-3-flagged.csv"

df = pd.read_excel(INPUT_FILE)

COL = "Delay_Reason"
s = df[COL].astype(str)

clean = (
    s.str.lower()
     .str.replace(r"[^\w\s]", "", regex=True)    
     .str.replace(r"\s+", " ", regex=True)       
     .str.strip()
)

raw = s.str.strip()
only_punct_or_numbers = raw.str.match(r'^(?=.*\S)(?!.*[A-Za-z]).*$', na=False)

df.loc[only_punct_or_numbers, COL] = "0"

not_late_phrases = [
    "no delay", "not delay", "not delayed", "not late",
    "na", "0", "null", "nan"
]

def phrase_to_token_pattern(p: str) -> str:
    p = p.strip().lower()
    esc = re.escape(p).replace(r"\ ", r"\s+")
    return rf"(?<!\w){esc}(?!\w)"

pattern = r"(?:{})".format("|".join(phrase_to_token_pattern(p) for p in not_late_phrases))
regex = re.compile(pattern, flags=re.IGNORECASE)

phrase_hit = clean.str.contains(regex, na=False)

df["Reason_Is_Late"] = np.where(only_punct_or_numbers | phrase_hit, 0, 1)

df[COL] = clean
df.loc[only_punct_or_numbers, COL] = "0"

# Save
df.to_excel(OUTPUT_FILE, index=False)
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print("Saved:")
print(f" - {OUTPUT_FILE}")
print(f" - {OUTPUT_CSV}")
print(df[[COL, "Reason_Is_Late"]].head(20))

## Encode Categorical Variables